In [1]:
from flask import Flask, render_template, request
import cv2
import matplotlib.pyplot as plt
import numpy as np
import keras.models
import re
import sys
import os
sys.path.append(os.path.abspath('./model'))
import urllib


Using TensorFlow backend.


In [2]:
#init flask app

app = Flask(__name__)
model = keras.models.load_model('model/italianet_keras_v1.h5')

@app.route('/')
def index():
    return render_template('index.html')



In [3]:

def process_img(link):
    urllib.request.urlretrieve(link, "input.jpg")
    return "input.jpg"

In [ ]:
@app.route('/predict', methods=['GET','POST'])
def imgpath_to_plot():
    #if you dont have this file yet
    #test_data = process_test_data()
    link = request.get_data().decode("utf-8")
    print(link)
    urllib.request.urlretrieve(link, "input.jpg")
    img = cv2.imread("input.jpg")

    orig = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    im = cv2.resize(orig,(128,128))#resize it to pass it into the neural network
    im = np.reshape(im,(1,128,128,3))/255 #normalize (divide by 255)
    
    model_out = model.predict([im])[0]
        
    pizzaness = round(model_out[1]*100, 2)

    if pizzaness >= 50:
        str_label = '{}% Pizza'.format(pizzaness)
    else:
        str_label = '{}% Calzone'.format(100-pizzaness)

    plt.imshow(orig,interpolation='nearest')
    plt.title(str_label, x=0.5, y=0.85, fontsize=24, bbox = dict(facecolor='white', alpha=0.5))
    plt.axis('off')
    plt.savefig('out.jpg')
    return str_label

In [ ]:
if __name__ == '__main__':
    port = int(os.environ.get('PORT',5000))
    app.run(host='0.0.0.0',port=port)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


https://www.cicis.com/media/1243/pizza_adven_zestypepperoni.png


127.0.0.1 - - [10/Dec/2017 19:11:02] "POST /predict HTTP/1.1" 200 -


https://images-gmi-pmc.edge-generalmills.com/9df0ff18-e881-4221-ac91-710b37668b03.jpg


127.0.0.1 - - [10/Dec/2017 19:11:24] "POST /predict HTTP/1.1" 200 -


https://atmedia.imgix.net/f95b20c40c8e2cedc27ee73ab9c567d8babb33c4?w=1500&fit=max


127.0.0.1 - - [10/Dec/2017 19:11:55] "POST /predict HTTP/1.1" 200 -


In [ ]:
imgpath_to_plot('https://www.cicis.com/media/1243/pizza_adven_zestypepperoni.png')